In [1]:
import os
import sys
import h5py
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

from pynwb import NWBFile, NWBHDF5IO
from pynwb.core import DynamicTable
from pynwb.image import ImageSeries
from pynwb.misc import AnnotationSeries
from pynwb.behavior import SpatialSeries, Position, BehavioralTimeSeries, BehavioralEvents
from ndx_multichannel_volume import CElegansSubject, OpticalChannelReferences, OpticalChannelPlus, ImagingVolume, MultiChannelVolume, MultiChannelVolumeSeries, SegmentationLabels
from pynwb.ophys import ImageSegmentation, PlaneSegmentation, DfOverF, RoiResponseSeries, Fluorescence

In [2]:
def resolve_worm_id(root):
    id = root.split('\\')

    meta = id[len(id)-1].split('-')
    date = meta[1]
    sex = sex_map[meta[2][0]]
    num = meta[2][1:]
    
    return date, sex, num
    
def read_activity(path, date, num):
    traces = pd.read_csv(os.path.join(path, 'raw.csv'))

    traces['date'] = traces['date'].astype(str)
    traces['animal'] = traces['animal'].astype(str)

    target_frame = traces[traces['date'] == date]
    target_frame = target_frame[target_frame['animal'] == num]

    target_labels = target_frame[['neuron', 'neuron_group']]
    target_frame = target_frame.iloc[:, 9:].to_numpy()
    
    return target_frame, target_labels

In [3]:
master_path = Path("E:\\venkatachalam\\000981")
anno_path = Path("G:\\Northeastern University\\Seyedehmaedeh Seyedolmohadesin - sexual_dimorphism_data")

sex_map = {
    'h': 'hermaphrodite',
    'm': 'male'
}

count = 0

In [4]:
for root, dirs, files in os.walk(master_path):
    file_progress = tqdm(range(len(files)), leave=False)
    for n in file_progress:
        file = files[n]
        
        if file[-3:] == 'nwb':
            file_path = os.path.join(root, file) 
            date, sex, num = resolve_worm_id(root)
            file_progress.set_description(f"[{date}] Worm #{num} ({sex}) @{file_path[:3]}...\\{file}.")


            # Load neuron tracks & activity
            annotations = os.path.join(anno_path, date, sex, num, 'annotations.h5')
            worldlines = os.path.join(anno_path, date, sex, num, 'worldlines.h5')
            target_frame, target_labels = read_activity(anno_path, date, num)


            # Unpack neuron tracks
            with h5py.File(annotations, 'r') as h5_file:
                t = h5_file['t_idx'][:]
                wlid = h5_file['worldline_id'][:]
                x = h5_file['x'][:]
                y = h5_file['y'][:]
                z = h5_file['z'][:]

            with h5py.File(worldlines, 'r') as h5_file:
                id = h5_file['id'][:]
                name = h5_file['name'][:]


            # Open nwb file
            io = NWBHDF5IO(file_path, mode="r")
            nwbfile = io.read()


            # Cache existing objects in memory
            calc_imaging_vol = nwbfile.processing['NeuroPAL'].data_interfaces['TrackedNeurons'].plane_segmentations['TrackedNeuronROIs'].imaging_plane
            original_tracks = nwbfile.processing['NeuroPAL'].data_interfaces['TrackedNeurons'].plane_segmentations['TrackedNeuronROIs']


            # Remove existing objects in file
            nwbfile.processing['CalciumActivity'].data_interfaces.pop('TrackedNeurons')
            #nwbfile.processing['CalciumActivity'].data_interfaces.pop('ActivityTraces')
            nwbfile.processing['NeuroPAL'].data_interfaces.pop('TrackedNeurons')


            # Save neuron activity
            #activity_labels = DynamicTable(name='ActivityTraces', description='Neuron IDs associated with ActivityDfOverF', timestamps=[])
            #activity_labels.add_column(name='activity', description='neuron activity')
            #activity_labels.add_column(name='neuron', description='neuron name')
            #activity_labels.add_column(name='neuron_group', description='neuron group')
            #for idx, row in target_labels.iterrows():
            #    activity_labels.add_row({'activity': np.nan_to_num(target_frame[idx]), 'neuron': row['neuron'], 'neuron_group': row['neuron_group']})


            # Create neuron track objects...
            NeuroPALTracks = ImageSegmentation(
                name='TrackedNeurons',
            )


            trackIDs = PlaneSegmentation(
                name='TrackedNeuronROIs',
                description='Neuron centers as tracked throughout GCaMP video. Formatted as (x, y, z, t)',
                imaging_plane=calc_imaging_vol,
            )


            # ...add ROIs...
            for eachNeuron in range(len(wlid)):
                trackIDs.add_roi(voxel_mask=[[x[eachNeuron], y[eachNeuron], z[eachNeuron], t[eachNeuron]]])


            # ...add neuron IDs...
            label = []
            for eachNeuron in range(len(wlid)):
                label.append(name[wlid[eachNeuron]])

            trackIDs.add_column(
                name='TrackedNeuronIDs',
                description='Neuron ID labels for tracked neuron rois.',
                data=label,
                index=False,
            )

            # ...save to parent object
            NeuroPALTracks.add_plane_segmentation(trackIDs)


            # Validate unchanged properties
            #assert original_tracks.imaging_plane == NeuroPALTracks.imaging_plane


            # Save objects to cached nwbfile.
            nwbfile.processing['CalciumActivity'].add(NeuroPALTracks)
            #nwbfile.processing['CalciumActivity'].add_container(activity_labels)

            # Save changes to file.
            nwbfile.set_modified()

            with NWBHDF5IO(os.path.join(master_path.parent, 'fixed', file), mode='w') as export_io:
                export_io.export(src_io=io, nwbfile=nwbfile)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1009.70it/s]
[20220327] Worm #2 (hermaphrodite) @E:\...\sub-20220327-h2_ses-20220327_ophys.nwb.:  64%|███████▋    | 7/11 [08:44<04:59, 74.87s/it]


KeyboardInterrupt: 